In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Pride and Prejudice.txt to Pride and Prejudice.txt


In [ ]:
file = open("Pride and Prejudice.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

'Once upon a time in a small village nestled between rolling hills and lush forests, there lived a young girl named Lily. Lily was known throughout the village for her kindness and adventurous spirit. Every day after finishing her chores, she would wander into the woods, exploring new paths and discovering hidden treasures. One sunny morning, while wandering deeper into the forest than usual, Lily stumbled upon a mysterious old cottage hidden behind a thicket of brambles. Curiosity piqued, she ca'

In [ ]:
len(data)

2356

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:5]

220
The Length of sequences are:  385
Data:  [[24 20  1]
 [20  1 50]
 [ 1 50  8]
 [50  8  1]
 [ 8  1 51]
 [ 1 51 25]
 [51 25 52]
 [25 52 53]
 [52 53 54]
 [53 54 55]]
Response:  [50  8  1 51 25 52 53 54 55  3]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             2200      
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 220)               220220    
                                                                 
Total params: 13271420 (50.63 MB)
Trainable params: 13271420 (50.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
7/7 [==============================] - ETA: 0s - loss: 5.3884
Epoch 1: loss improved from inf to 5.38837, saving model to next_words.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 9s 529ms/step - loss: 5.3884
Epoch 2/70
7/7 [==============================] - ETA: 0s - loss: 5.3197
Epoch 2: loss improved from 5.38837 to 5.31966, saving model to next_words.h5
7/7 [==============================] - 4s 534ms/step - loss: 5.3197
Epoch 3/70
7/7 [==============================] - ETA: 0s - loss: 5.1249
Epoch 3: loss improved from 5.31966 to 5.12491, saving model to next_words.h5
7/7 [==============================] - 5s 700ms/step - loss: 5.1249
Epoch 4/70
7/7 [==============================] - ETA: 0s - loss: 5.1306
Epoch 4: loss did not improve from 5.12491
7/7 [==============================] - 3s 465ms/step - loss: 5.1306
Epoch 5/70
7/7 [==============================] - ETA: 0s - loss: 5.1589
Epoch 5: loss did not improve from 5.12491
7/7 [==============================] - 3s 451ms/step - loss: 5.1589
Epoch 6/70
7/7 [==============================] - ETA: 0s - loss: 4.9922
Epoch 6: loss improved from 5.12491 to 4.99219, savin

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  print(predicted_word)
  return predicted_word
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)

          Predict_Next_Words(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue

['turned', 'into', 'weeks']
1/1 [==============================] - 1s 831ms/step
lily
['excitement', 'and', 'nervousness']
1/1 [==============================] - 0s 80ms/step
and
['Determined', 'to', 'help']
1/1 [==============================] - 0s 44ms/step
lily
